<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/Widen00-Lab1/Lab1/Widen00_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DD2363 Lab 1: Matrix Factorization**
**Joel Widén**

# **Abstract**

This report is investigating different ways to directly solve matrix equations. A big part of this is factorizing matrices into forms that are more easily calculated or inverted. In this report the methods used are sparse matrix matrix vector multiplication and QR factorization. These methods are used and tested according to stated test cases. All the methods performed as expected which was expected. These methods allows for either faster calculations such as the sparse matrix multiplication or clever ways of dealing with direct matrix equations where the matrix might not be easily invertible.

#**About the code**

This is a report about matrix factorization in the course DD2363 Methods in Scientific Computing. The author of this file is Joel Widén, joelwid@kth.se.

# **Set up environment**


This block is run to set up the environment.


In [16]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Numbered algorithms, equations and chapter references used in this report is from the DD2363 course book [Methods in Computational Science by Johan Hoffman](https://epubs.siam.org/doi/book/10.1137/1.9781611976724) if not stated otherwise.

This report is investigating different methods of solving the direct matrix equation
$Ax = b$ (eq 5.1)
 by first implementing matrix factorization methods so that $x = A^{-1}b$ can be solved. This is split into several different assignments with the required output, input and test method of the function:

**Assignment 1:** Function: sparse matrix-vector product

  * *Input:* vector x, sparse (real, quadratic) matrix A: CRS arrays val, col_idx, row_ptr

  * *Output:* matrix-vector product $b=Ax$

  * *Test:* verify accuracy against dense matrix-vector product.

**Assignment 2:** Function: QR factorization

  * *Input:* (real, quadratic, invertible) matrix $A$

  * *Output:* orthogonal matrix $Q$, upper triangular matrix $R$, such that $A=QR$

  * *Test:* $R$ upper triangular, Frobenius norms $|| Q^TQ-I ||_F$ and $|| QR-A ||_F$

**Assignment 3:** Function: direct solver $Ax=b$

  * *Input:* (real, quadratic) matrix $A$, vector $b$

  * *Output:* vector $x=A^-1b$

  * *Test:* residual $|| Ax-b ||$, and $|| x-y ||$ where $y$ is a manufactured solution with $b=Ay$

To complete assignment 1 algorithm 5.9 from the course book is implemented to calculate the matrix. This algorithm is chosen as it does exactly what is asked from the question as it calculates a sparse matrix vector product.

Assignment 2 was completed using algorithm 5.3 which is a modified gram-schmidt agorithm. This is used as the matrix A is factorized as a QR factorization as this specific algorithm does. It outputs Q as a orthogonal matrix and R as an upper triangular matrix.

Assignment 3 is solved using the QR-factorization from the previous assignment combined with algorithm 5.2 to solve the resulting problem which involves an upper triangular matrix.


# **Method**

**Assignment 1: Sparse matrix vector product**
The sparse matrix algorithm uses the fact that the majority of the entries are zeroes and uses a clever system of pointers and id:s to deconstruct the information of a matrix in a more compact way. This data structure is called CRS or compressed row storage which is described in chapter 5.8. The algorithm can then utilize the fact of the rest of the entries being zeros to perform the matrix vector multiplication by using the CRS data. This is implemented in this assignment.

In [17]:
#Sparse matrix

#Sparse matrix vector product based on Algorithm 5.9
def sparse_matrix_vector_product(spA, x):
  n = max(spA.col_idx)
  b = np.zeros(n, dtype="int")
  for i in range(0, n):
    for j in range(spA.row_ptr[i]-1, spA.row_ptr[i+1]-1):
      b[i] = b[i] + spA.val[j]*x[spA.col_idx[j]-1]
  return b

x_1 = np.array([1, 1, 1, 1, 1, 1])
# From example 5.5
# Construct a simple sparse matrix class using the CRS data structure
class spMatrix:
  def __init__(self, val, col_idx, row_ptr):
    self.val = val
    self.col_idx = col_idx
    self.row_ptr = row_ptr

# Create a sparse matrix object
val = np.array([3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3])
col_idx = np.array([1, 2, 4, 2, 3, 3, 3, 4, 5, 5, 6])
row_ptr = np.array([1, 4, 6, 7, 9, 10, 12])
spA = spMatrix(val, col_idx, row_ptr)

# Creating matrix for comparison dense calculation
A_sparse = np.array([[3, 2, 0, 2, 0, 0], [0, 2, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 3, 2, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 2, 3]])

#Calling function
b_sparse = sparse_matrix_vector_product(spA, x_1)
b_dense = A_sparse.dot(x_1)

**Assignment 2: QR-factorization**

QR factorization is a way of factorizing a square matrix $A$ into an orthogonal matrix Q and an upper triangular matrix $R$ such that $A = QR$ (chapter 5.3). This is useful as the inverse of a orthogonal matrix is the same as its transverse according to chapter 2.6. So $Q^T = Q^{-1}$. The upper trianguar matrix R has some useful properties which will be explained in the following assignment.


In [18]:
#QR factorization. Want to use modified grahm-schmidt algorithm

#Algorithm 5.3
def modified_gram_schmidt_iteration(A):
  n = len(A)
  R = np.zeros((n,n))
  Q = np.zeros((n,n))
  for j in range(0, n):
    v = A[:,j]
    for i in range(0, j):
      R[i][j] = Q[:,i].dot(v)
      v = v - R[i][j]*Q[:,i]
    R[j][j] = np.linalg.norm(v)
    Q[:,j] = v/R[j][j]
  return(Q, R)

# Vector for QR factorization and direct solving
A_inv = np.array([[2, -1], [-1, 2]])
#Call function
Q, R = modified_gram_schmidt_iteration(A_inv)
frob_1 = np.linalg.norm(Q.transpose().dot(Q) - np.identity(len(Q)), "fro")
frob_2 = np.linalg.norm(Q.dot(R) - A_inv, "fro")

**Assignment 3: Direct solver**
To perform this calculation the factorized matrix from assignment 2 is used. This allows a rewrite of $Ax=b$ as

$QRx = b ⇒ Rx = Q^Tb$

since $QQ^T = I$ according to the definition of orthogonal matrices. This is a new equation which can be solved for $x$ by implementing a backward substitution algorithm since $R$ is an upper triangular matrix.

In [19]:
# Direct Solver. Previous block must be run before this one.
# Direct Solver will be using QR factorization for inverting A using Q and R
# from previous assignment. Q is orthogonal and its inverse is equal to Q^T.
# Using backward substitution to solve R*x = Q^T*b as Ax = b => QRx = b

#Backwards substitution from algorithm 5.2
def backward_substitution(U,b):
  n = len(b)
  x = np.zeros(n)
  x[n-1] = b[n-1]/U[n-1,n-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i+1, n-2, -1):
      sum = sum + U[i][j]*x[j]
    x[i] = (b[i] - sum)/U[i][i]
  return x

#Function calling and vector definitions
y = np.array([5, 3])
b = A_inv.dot(y)
QTb = Q.transpose().dot(b)
x = backward_substitution(R, QTb)
residual = np.linalg.norm(A_inv.dot(x)-b)
error = np.linalg.norm(x-y)

# **Results**

The result from the code is presented by running the code block below.

In [20]:
# Results assignment 1
print("Assignment 1")
if (b_sparse == b_dense).all():
  print("Question 1 verified with dense matrix")
print("Sparse matrix computated b =", b_sparse)
print("Dense matrix computated b =", b_dense)

# Results assignment 2
print("\n" + "Assignment 2")
print("Frobenius norm for Q^T*Q - I = ",frob_1)
print("Frobenius norm for Q*R - A = ",frob_2)
print("Q = ", Q)
print("R = ", R)

# Results assignment 3
print("\n" + "Assignment 3")
print("Solution x =", x)
print("Manufactured solution y =", y)
print("Residual: ", residual)
print("Error: ", error)

Assignment 1
Question 1 verified with dense matrix
Sparse matrix computated b = [7 3 1 5 1 5]
Dense matrix computated b = [7 3 1 5 1 5]

Assignment 2
Frobenius norm for Q^T*Q - I =  2.6901577681355055e-16
Frobenius norm for Q*R - A =  0.0
Q =  [[ 0.89442719  0.4472136 ]
 [-0.4472136   0.89442719]]
R =  [[ 2.23606798 -1.78885438]
 [ 0.          1.34164079]]

Assignment 3
Solution x = [5. 3.]
Manufactured solution y = [5 3]
Residual:  2.6645352591003757e-15
Error:  1.9860273225978185e-15


# **Discussion**

From the first assignment the matrix vector product is exactly the same as the dense matrix vector product. This is expected since the operations is identical and only carried out in different ways. The sparse matrix product is vastly more efficient than the dense matrix product though as the complexity of the sparse matrix is $\mathcal{O}(n)$ compared to the dense matrix multiplication which is of order $\mathcal{O}(n^2)$ according to chapter 5.8.

The second assignment performs the QR factorization of a matrix which allows the original matrix $A$ to ber more easily inverted and therefore solved in matrix equations. Both frobenius norms $|| Q^TQ-I ||_F$ and $|| QR-A ||_F$ are close or equal to zero which means the algorithm outputs $Q$ as an orthogonal matrix and the second norm that $A = QR$. One of the norms is non-zero due to float error. The matrix $R$ is also upper triangular which should be the result of a QR factorization.

The last assignment is using the QR factorization to solve the direct equation. The residual and error in this assignment is close to zero and should theoretically be zero but due to float error is not exactly zero. Comparing the outputted vectors reveals that $x$ is the same as the manufactured solution $y$.